## pg_restore cipsrv_owld

In [ ]:
import os,sys,psycopg2,argparse;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
from contextlib import closing;
parser = argparse.ArgumentParser();
parser.add_argument("--owld_dumpfiles"   ,required=False);
parser.add_argument("--use_existing"     ,required=False,default=False,action=argparse.BooleanOptionalAction);
parser.add_argument("--override_username",required=False);
args, _ = parser.parse_known_args();

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

s3bk = os.environ['S3_BUCKET'] + 'owld/';
# override: s3bk = 'https://edap-ow-data-commons.s3.amazonaws.com/data/cipsrv/owld/';
lddk = os.path.join(os.path.expanduser('~'),'loading_dock');
dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pswd = os.environ['POSTGRESQL_CIP_PASS'];
thds = 2;

if args.owld_dumpfiles is None:
   srcs = 'WQP:cipsrv_owld_wqp_20250221.dmp,'       \
        + 'ATTAINS:cipsrv_owld_attains_20250126.dmp'\
        + 'NPDES:cipsrv_owld_npdes_20250502.dmp'    \
        + 'FRSPUB:cipsrv_owld_frspub_20250502.dmp';
else:
   srcs = args.owld_dumpfiles;

component_name = 'cipsrv_owld';
component_type = 'dataset';
ary_src = srcs.split(',');
for item in ary_src:
    (pref,src)=item.split(':');
    print(pref + " source: " + s3bk + src);


In [ ]:
for item in ary_src:
    (pref,src)=item.split(':');
 
    if args.use_existing:
        print("using existing " + src);
    
    else:
        z = common.download(
            s3  = s3bk
           ,src = src
           ,ld  = lddk
           ,trg = src
        );
    
        if not os.path.exists(os.path.join(lddk,src)):
            raise Exception('failed to download dmp file');
    
        print("successfully downloaded " + src);
    

In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
for item in ary_src:
    (pref,src)=item.split(':');
    
    with closing(conn.cursor()) as cursor:
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_attr         CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_cip          CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_cip_geo      CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_control      CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_huc12        CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_huc12_geo    CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_rad_a        CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_rad_evt2meta CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_rad_l        CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_rad_metadata CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_rad_p        CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_rad_srccit   CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_sfid         CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_src2cip      CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_src_a        CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_src_l        CASCADE");
        cursor.execute("DROP TABLE IF EXISTS cipsrv_owld." + pref + "_src_p        CASCADE");
        conn.commit();
    
    print("preexisting " + pref + " data purged.");

In [ ]:
for item in ary_src:
    (pref,src)=item.split(':');
    
    z = common.pg_restore(
         host
        ,port
        ,dbse
        ,user
        ,pswd
        ,os.path.join(lddk,src)
        ,thds
    );
    
    print("pg_restore for " + pref + " complete.");
    

In [ ]:
if z[0] == b'':
    a = common.upsert_registry(conn,component_name,component_type,src,args.override_username);
else:
    print(str(z));
    sys.exit(-10);
